# Kubernetes RAG System - Example Usage

This notebook demonstrates how to use the Kubernetes RAG system for learning and testing.

## 1. Setup and Configuration

In [ ]:
import sys
sys.path.append('..')

from src.utils.config_loader import get_config
from src.ingestion.pipeline import create_ingestion_pipeline
from src.retrieval.retriever import create_retriever
from src.generation.llm import create_rag_generator

# Load configuration
config, settings = get_config()
print("✓ Configuration loaded successfully")

## 2. Create Components

In [ ]:
# Create ingestion pipeline
pipeline = create_ingestion_pipeline(config)

# Create retriever
retriever = create_retriever(config)

# Create generator
generator = create_rag_generator(config)

print("✓ All components created successfully")

## 3. Ingest Sample Kubernetes Documentation

In [ ]:
sample_doc = """
# Kubernetes Fundamentals

## Pods

<details>
<summary>What is a Pod?</summary><br><b>
A Pod is the smallest deployable unit in Kubernetes. It represents a single instance of a running process in your cluster and can contain one or more containers.
</b></details>

<details>
<summary>Can a Pod contain multiple containers?</summary><br><b>
Yes, a Pod can contain multiple containers. These containers share the same network namespace and can communicate via localhost.
</b></details>

## Services

<details>
<summary>What is a Kubernetes Service?</summary><br><b>
A Service is an abstract way to expose an application running on a set of Pods as a network service. It provides a stable IP address and DNS name.
</b></details>

## Deployments

<details>
<summary>What is a Deployment?</summary><br><b>
A Deployment provides declarative updates for Pods and ReplicaSets. It allows you to describe the desired state and the Deployment controller changes the actual state to the desired state.
</b></details>
"""

# Ingest the sample document
num_chunks = pipeline.ingest_from_text(
    text=sample_doc,
    metadata={'source': 'notebook_example', 'type': 'tutorial'},
    source_name='k8s_fundamentals'
)

print(f"✓ Ingested {num_chunks} chunks")

## 4. Simple Query

In [ ]:
query = "What is a Pod in Kubernetes?"

# Retrieve relevant documents
results = retriever.retrieve(query, top_k=3)

print(f"Query: {query}\n")
print(f"Retrieved {len(results)} documents:\n")

for i, result in enumerate(results, 1):
    print(f"{i}. [Score: {result['score']:.3f}]")
    print(f"   {result['content'][:150]}...\n")

## 5. Generate Answer

In [ ]:
# Generate answer using LLM
answer_data = generator.generate_answer(
    query=query,
    retrieved_docs=results,
    temperature=0.3
)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(answer_data['answer'])
print("=" * 80)
print(f"\nBased on {answer_data['num_sources']} source(s)")

## 6. Multiple Queries

In [ ]:
queries = [
    "What is a Kubernetes Service?",
    "Explain Deployments in Kubernetes",
    "Can a Pod have multiple containers?"
]

for query in queries:
    results = retriever.retrieve(query, top_k=2)
    answer_data = generator.generate_answer(query, results)
    
    print(f"\n{'='*80}")
    print(f"Q: {query}")
    print(f"{'='*80}")
    print(f"A: {answer_data['answer']}")

## 7. Conversational Mode

In [ ]:
conversation_history = []

queries_conversation = [
    "What is a Pod?",
    "How is it different from a container?",
    "Can you give an example?"
]

for query in queries_conversation:
    results = retriever.retrieve(query, top_k=3)
    
    answer_data = generator.generate_with_followup(
        query=query,
        retrieved_docs=results,
        conversation_history=conversation_history
    )
    
    conversation_history = answer_data['conversation_history']
    
    print(f"\nUser: {query}")
    print(f"Assistant: {answer_data['answer']}")
    print("-" * 80)

## 8. Category-based Retrieval

In [ ]:
# Retrieve only Q&A pairs
qa_results = retriever.retrieve_by_category(
    query="What is a Deployment?",
    category="qa_pair",
    top_k=3
)

print("Q&A Pairs Found:\n")
for i, result in enumerate(qa_results, 1):
    print(f"{i}. [Score: {result['score']:.3f}]")
    print(f"   Type: {result['metadata'].get('type')}")
    print(f"   {result['content'][:200]}...\n")

## 9. System Statistics

In [ ]:
from src.retrieval.vector_store import VectorStore

vector_store = VectorStore(
    collection_name=config.vector_db.collection_name,
    persist_directory=config.vector_db.persist_directory
)

stats = vector_store.get_collection_stats()

print("📊 System Statistics")
print("=" * 50)
print(f"Collection: {stats['name']}")
print(f"Total Documents: {stats['count']}")
print(f"Storage Location: {stats['persist_directory']}")
print("=" * 50)

## 10. Cleanup (Optional)

In [ ]:
# Uncomment to reset the database
# vector_store.delete_collection()
# print("✓ Database reset complete")